In [30]:
from datasets import load_dataset
dataset = load_dataset('pacovaldez/stackoverflow-questions', split='validation')

Found cached dataset parquet (/home/koechian/.cache/huggingface/datasets/pacovaldez___parquet/pacovaldez--stackoverflow-questions-e77b48110dd75272/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [31]:
from sentence_transformers import SentenceTransformer 
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

In [32]:
texts = dataset['title']
texts[:5]

['How to dynamically instntiate a class?',
 'update value on button on:click event vue js',
 'Setinterval for div in jquery',
 'Phonegap twitter plugin for ios4',
 'Looping thru 2 set of data in puppet - optimalization']

In [33]:
sample = texts[8]
print(model.encode(sample))

[ 1.85674667e-01 -3.08939934e-01 -9.86766219e-01 -4.18468267e-01
 -1.13898718e+00 -6.24335289e-01  4.17806864e-01  4.11969274e-01
 -2.51463294e-01  3.24652106e-01  9.60822925e-02 -2.80396134e-01
  1.00697979e-01  9.27737653e-02  1.62881672e-01 -4.76988629e-02
  1.90256357e-01  2.14954346e-01  1.55171469e-01 -1.27271444e-01
 -1.24332249e-01  4.38459069e-01 -1.89027235e-01  7.32985914e-01
 -1.26352519e-01 -4.39275503e-01  7.09219515e-01 -5.36065996e-01
 -5.05016506e-01 -3.17519486e-01 -6.70124888e-01 -6.27895832e-01
  7.10475564e-01  1.28862947e-01  4.51263152e-02  9.92517352e-01
  4.73340213e-01  6.42844010e-03  1.35540053e-01 -1.71526060e-01
  2.62077928e-01 -3.17243099e-01 -3.16399261e-02  3.87279363e-03
  6.34526253e-01  8.94257873e-02 -3.27702284e-01  2.59590209e-01
 -1.19833238e-01  3.76557410e-01 -6.39917254e-01  3.98923099e-01
 -1.26914293e-01 -5.97151816e-02 -3.34018797e-01 -1.43960491e-01
  1.81257814e-01  7.54507557e-02  2.19950266e-03 -8.36942494e-02
  6.94525763e-02 -5.18321

In [34]:
features = model.encode(texts[:10000])


## Clustering using Kmeans

In [35]:
from sklearn.cluster import KMeans

cl = KMeans(n_clusters = 10)
cl.fit(features)

cluster_labels = cl.labels_

/home/koechian/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [36]:
from sklearn.metrics.pairwise import cosine_similarity

def compare(query, texts, features):
    encoded_query = model.encode([query][0])
    scores = cosine_similarity([encoded_query],features)[0]
    results = [(texts[i],scores[i]) for i in range(len(texts))]
    results.sort(key=lambda x:x[1],reverse=True)

    return results

In [56]:

import pandas as pd 

def deploy(error):
    results = compare(error,texts[:10000],features)
    scores = []
    titles = []

    for i in range(len(results)):
        titles.append(results[i][0])
        scores.append(round(results[i][1]*100,2))

    df  = pd.DataFrame({'Described Error':titles,'Similarity Score (%)':scores})
    return df.head()

In [57]:
deploy('how to center a div')

,Described Error,Similarity Score (%)
0,How to center image in a div horizontally and ...,81.86
1,Align text to center in div with another div a...,77.74
2,"Keep text on left and middle of the div, not c...",74.61
3,Center a H1 tag inside a DIV,73.66
4,How do I center the placeholder and text insid...,61.36
